In [1]:
print("Hello LangChain")

Hello LangChain


In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

#load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY[:10])
print(OPENAI_API_KEY[:5])

gsk_3


In [20]:
# prompt + llm + output 

# prompt: (쉽게 말해) AI에게 하는 질문
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , # AI에게 역할을 주는 것.
     ("user", "{input}") ]
)
print(type(prompt))
print(prompt)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [30]:
prompt_text = prompt.format(input="파이썬은 무엇인가요? 자세하게 설명해주세요")
print(type(prompt_text))
print(prompt_text)

<class 'str'>
System: 당신은 개발자입니다.
Human: 파이썬은 무엇인가요? 자세하게 설명해주세요


In [32]:
#llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125")

# Groq는 ChatGPT와 호환 -> ChatGPT 모델 사용 가능
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성 (모델 가져와서 생성)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct", # 사용 모델: llama-4-scout
    model="moonshotai/kimi-k2-instruct-0905", # 사용 모델: kimi-k2
    temperature=0.7
)
print(type(llm)) # 타입: ChatOpenAI
print(llm)

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
client=<openai.resources.chat.completions.completions.Completions object at 0x14029dd50> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x14029d790> root_client=<openai.OpenAI object at 0x14029f3d0> root_async_client=<openai.AsyncOpenAI object at 0x14029cf50> model_name='moonshotai/kimi-k2-instruct-0905' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [33]:
# 생성한 모델에 질문하기(prompt 입력)
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
응답: 파이썬(Python)은 1991년 네덜란드의 개발자 귀도 반 로섬(Guido van Rossum)이 만들고 1994년 1.0으로 공개된 고급 프로그래밍 언어입니다. 이름은 그가 좋아하는 코미디 그룹 ‘Monty Python’에서 따왔습니다. 다음은 파이썬이 어떤 언어이며, 어떤 특징을 가지고 있는지, 어떻게 활용되는지를 가능한 한 상세히 정리한 설명입니다.

────────────────────────────────────────
1. 언어적 위치와 철학
────────────────────────────────────────
1) 고급 언어(High-level language)
- 하드웨어 제어에 가까운 ‘저급’ 언어(C, Assembly)와 달리 사람이 읽고 쓰기 쉽게 설계됨
- 메모리 관리, 포인터 연산 같은 저수준 세부사업을 언어 차원에서 자동 처리

2) 인터프리터 방식(Interpreter)
- 소스 코드(.py)를 한 줄씩 즉시 해석해 실행. 컴파일 언어(C, C++, Rust 등)처럼 빌드-링크-실행 단계가 필요 없음
- 개발-테스트 사이클이 짧고 대화형(REPL) 환경이 기본 제공 → 실험적 코딩·데이터 탐색에 적합

3) “배터리 포함(Batteries Included)” 철학
- 표준 라이브러리만으로도 파일 압축·이메일·웹 서버·데이터베이스·암호화 등을 해결할 수 있도록 풍부한 모듈 포함
- 외부 라이브러리가 있으면 더욱 강력하지만, “무작정 깔기 전에” 표준만으로도 충분한 경우가 많음

4) 가독성과 명확성
- PEP 20(“Zen of Python”)에 ‘읽기 쉬운 코드가 쓰기 쉬운 코드보다 중요하다’는 취지 담김
- 들여쓰기(Indentation)로 블록 구조를 강제 → 작성자의 의도가 시각적으로 드러남

────────────────────────────────────────
2. 언어적 특징
────────────────────────────

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser
print(type(chain))
print(chain)

<class 'langchain_core.runnables.base.RunnableSequence'>
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x13794dcd0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x140224090>, root_client=<openai.OpenAI object at 0x1378a6f90>, root_async_client=<openai.AsyncOpenAI object at 0x137e7d6d0>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')] last=StrOutputParser()


In [9]:
response = chain.invoke({"input":"지구의 자전주기는 얼마인가요?"})

In [10]:
print(type(response))
print(response)

<class 'str'>
지구의 자전주기는 24시간입니다. 이를 태양시라고 부르기도 합니다.
